# Differential Privacy for PRETSA

In [ ]:
!git clone https://github.com/cmalvr/PRETSA

Cloning into 'PRETSA'...
remote: Enumerating objects: 256, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 256 (delta 6), reused 7 (delta 2), pack-reused 240 (from 1)
Receiving objects: 100% (256/256), 11.46 MiB | 10.49 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [ ]:
!pip install -r /content/PRETSA/requirements.txt

  Preparing metadata (setup.py) ... done
  Created wheel for anytree: filename=anytree-2.4.3-py2.py3-none-any.whl size=26707 sha256=94e8b2a9daee3c9c0a694f4f0683e30d0ccc9d044f06180e63ea206d1d294b57
  Stored in directory: /root/.cache/pip/wheels/f3/d3/8f/9eea88a7fe27ca18e7ebfc23e1012514c8b13a4552aa43be12
Successfully built anytree


In [37]:
from pathlib import Path
import zipfile

# Define paths
zip_path = Path("/content/PRETSA/baselogs.zip")
extract_dir = Path("/content/PRETSA")

# Extract zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Search for CSV files
csv_files = list(extract_dir.rglob("*.csv"))

if csv_files:
    print("Found CSV files:", [f.name for f in csv_files])
else:
    print("No CSV files found.")

Found CSV files: ['bpic2013_dataset.csv', 'Sepsis_dataset.csv', 'traffic_fines_dataset.csv', 'HospitalBilling_dataset.csv', 'CoSeLoG_dataset.csv']


In [ ]:
!python /content/PRETSA/dataset_collection.py

## Running Annotations for three datasets

In [50]:
# Generates all the original annotation duration for each of the three datasets: CoSeLog, Sepsis and Road Traffc
# Generates /content/PRETSA/original_annotation/
# Seems to be the same as what is contained in baselogs but we will run in for sanity
!python /content/PRETSA/gc_add_annotations_duration.py with

Processing: CoSeLoG
/content/PRETSA/original_annotation/CoSeLoG_duration.csv
Processing: Sepsis
/content/PRETSA/original_annotation/Sepsis_duration.csv
Processing: Road_Traffic_Fine_Management_Process
/content/PRETSA/original_annotation/Road_Traffic_Fine_Management_Process_duration.csv


## Run Pretsa: Generates Pretsa Log


In [51]:
!python /content/PRETSA/startExperimentsForJournalExtension_PRETSA.py
# Generates: /content/PRETSA/pretsalog/ f"{dataset}_t{t}_k{k}_pretsa.csv"
# These contain the new annotations for duration after Running PRETSA method

 Load Event Log: /content/PRETSA/original_annotation/CoSeLoG_duration.csv
Generated Distance Matrix
 Modified 19 cases for k=4, t=1.0
/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=16 observations were given.
  return hypotest_fun_in(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=14 observations were given.
  return hypotest_fun_in(*args, **kwds)
 Private event log saved at: /content/PRETSA/pretsalog/CoSeLoG_t1.0_k4_pretsa.csv
 Experiment metadata saved at: /content/PRETSA/pretsalog/CoSeLoG_t1.0_k4_pretsa.pickle
 Load Event Log: /content/PRETSA/original_annotation/CoSeLoG_duration.csv
Generated Distance Matrix
 Modified 35 cases for k=8, t=1.0
/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:418: UserWarn

## Run Baseline: Generate Baseline log

In [ ]:
!python /content/PRETSA/generate_baseline_log.py <fileName> <k> <t>
# Generates: /content/PRETSA/baselinelogs/ Path(filePath).name.replace(".csv","_pretsa_baseline_k%s_t%s.csv" % (kString,tString))
# These contain the new annotations for duration after Running BASELINE method

/bin/bash: -c: line 1: syntax error near unexpected token `<'
/bin/bash: -c: line 1: `python /content/PRETSA/generate_baseline_log.py <fileName> <k> <t>'


## Calculate Statistics

In [ ]:
!python /content/PRETSA/calculateBaselineEventLogStatistics.py <dictName> #Needs: dictPath + dataset + "_duration_baseline_k" + str(k) + ".csv", generates: dictPath + "baseline_event_logs_statistics.csv"
!python /content/PRETSA/calculatePRETSAEventLogStatistics.py <dictName> #Needs: dictPath + dataset + "_duration_pretsa_k" + str(k) + ".csv", generates: dictPath + "pretsa_event_logs_statistics.csv"

## Generate Annotations

In [ ]:
!python /content/PRETSA/calculateAnnotationsEventLog_baseline.py <dictName> #Needs: filePath = dictPath + dataset + "_duration_pretsa_baseline_k" + str(k) + "_t" + str(t) + ".csv" ,Generates: dictPath + "pretsa_baseline_statistics_annotations.csv"
!python /content/PRETSA/calculateAnnotationsEventLog_pretsa.py <dictName> #Needs:dictPath + dataset + "_duration_t" + tString[t] + "_k" + str(k) + "_pretsa.csv" , generates dictPath + "pretsa_statistics_annotations.csv"
!python /content/PRETSA/generateAnnotationOriginalDataset.py <dictName> #generates original_annotations_pretsa.csv (located in <dictName>)

## Calculate Annotation Errors

 Relative error of the annotations for each activity

In [ ]:
!python /content/PRETSA/calculateAnnotationError.py <dictName> #Needs: original_annotations_pretsa.csv, pretsa_baseline_statistics_annotations.csv, pretsa_statistics_annotations.csv